# Tutorial 5: Instatiating a *scenario category* including I2V communication

In this tutorial, we will cover the following items:

1. Create *actor categories*, *activity categories*, and *physical thing categories*
2. Instantiate a *scenario category*
3. Create *actors*, *activities*, *events*, and *physical things*
4. Instantiate a *scenario*
5. Store the *scenario*

In [1]:
# Before starting, let us do the necessary imports
import os
import json
from domain_model import ActorCategory, ActivityCategory, Constant, Sinusoidal, \
    ScenarioCategory, StateVariable, PhysicalElementCategory, Tag, ActorType, \
    actor_category_from_json, scenario_category_from_json, Messages, \
    Activity, activity_from_json, ActivityCategory, Actor, \
    PhysicalElement, Scenario, scenario_from_json, State

## 1. Create *actor categories*, *activity categories*, and the *physical thing categories*

In this tutorial, we will create a *scenario category* in which the ego vehicle receives speed limit information from an I2V ITS network. The *scenario category* is depicted in the figure below. Here, the blue car represents the ego vehicle that is driving in a speed limit area and approaching an area with a new speed limit. The speed limit information is provided by 2 ITS road side units (RSUs). The ego vehicle changes speed according to the speed limits.

<img src="./examples/images/i2v.png" alt="I2V" width="400"/>

To create the *scenario category*, we first need to create the *actor categories*, *activity categories*, and the *physical things*. Let us start with the *actor categories*. The ego vehicle as well as I2V networks are defined by *actor categories*. Just like most objects, an *actor category* has a `name`, a `uid` (a unique ID), and `tags`. Additionally, an *actor category* has a `actor_type`.

Now let us create the *actor categories*. For this example, we assume that one *actor category* is a "vehicle" and both RSUs are "ITS Network" *actor categories*. Note that we can ignore the `uid` for now. When no `uid` is given, a unique ID is generated automatically. If no `tags` are provided, it will default to an empty list.

In [2]:
EGO_VEHICLE = ActorCategory(ActorType.Vehicle, name="EgoVehicle", 
                            tags=[Tag.EgoVehicle, Tag.RoadUserType_Vehicle,
                                  Tag.Communication_Network_Its])
RSU = ActorCategory(ActorType.NetworkIts, name="RSU",
                    tags=[Tag.Communication_Network_Its])

Now let us create the *activity categories* for the ego dynamic activities:

In [3]:
FOLLOWING_LANE = ActivityCategory(Constant(), StateVariable.LATERAL_ROAD_POSITION,
                                  name="Following lane",
                                  tags=[Tag.VehicleLateralActivity_GoingStraight])
DRIVING_FORWARD = ActivityCategory(Sinusoidal(), StateVariable.SPEED,
                                   name="driving forward",
                                   tags=[Tag.VehicleLongitudinalActivity_DrivingForward])

Now let us create the *activity category* for the I2V communication

In [4]:
RSU_MESSAGES = ActivityCategory(Messages(frequency=20, receiving=True), 
                                StateVariable.MESSAGE,
                                name="RSU_speed_limit",
                                tags=[Tag.Communication_I2V])

The last object we need to define before we can define the *scenario category* is the *static physical thing category*. A *scenario category* may contain multiple *physical things*, but for now we only define one that specifies the road layout. We assume that the scenario takes place at a straight motorway with multiple lanes:

In [5]:
MOTORWAY_CATEGORY = PhysicalElementCategory(description="Motorway with multiple lanes",
                                            name="Motorway",
                                            tags=[Tag.RoadLayout_Straight,
                                                  Tag.RoadType_PrincipleRoad])

## 2. Instantiate a *scenario category*

To define a *scenario category*, we need a description and a location to an image. After this, the static content of the scenario can be specified using the `set_physical_things` function. Next, to describe the dynamic content of the scenarios, the *actor categories* can be passed using the `set_actors` function and the *activity categories* can be passed using the `set_activities` function. Finally, using `set_acts`, it is described which activity is connected to which actor. 


In [6]:
I2V_CATEGORY = ScenarioCategory("./examples/images/i2v.png",
                                description="I2V speed limit information at the motorway",
                                name="I2V_CATEGORY")
I2V_CATEGORY.set_physical_elements([MOTORWAY_CATEGORY])
I2V_CATEGORY.set_actors([EGO_VEHICLE, RSU])
I2V_CATEGORY.set_activities([FOLLOWING_LANE, DRIVING_FORWARD, RSU_MESSAGES])
I2V_CATEGORY.set_acts([(EGO_VEHICLE, DRIVING_FORWARD), (EGO_VEHICLE, FOLLOWING_LANE),
                       (EGO_VEHICLE, RSU_MESSAGES)])

In [7]:
print(I2V_CATEGORY)

Name: I2V_CATEGORY
Description:
  I2V  speed  limit  information  at  the  motorway
Tags:
├─ EgoVehicle::ActorCategory
│  ├─ Communication_Network_Its
│  ├─ Communication_I2V
│  ├─ VehicleLateralActivity_GoingStraight
│  ├─ EgoVehicle
│  ├─ RoadUserType_Vehicle
│  └─ VehicleLongitudinalActivity_DrivingForward
├─ RSU::ActorCategory
│  └─ Communication_Network_Its
└─ Motorway::PhysicalElementCategory
   ├─ RoadLayout_Straight
   └─ RoadType_PrincipleRoad



## 3. Create *actors*, *activities*, *events*, and *static physical things*

We will create a *scenario* with the *scenario category* that we created in the previous chapters. The *scenario* is depicted in the figure below. Here, the blue car represents the ego vehicle driving in a 80 kph speed limit area. At the end of the 80 kph speed area the network quality decreases. The ego vehicle approaches a 100 kph speed limit area and accelerates from 80 to 100 kph.

<img src="./examples/images/i2v.png" alt="I2V" width="400"/>

Just as with a *scenario category*, we first need to create the components of a *scenario*. To create the *scenario*, we first need to create the *actors*, *activities*, *events*, and *physical things*.

### 3.1 Create *actors*

Now let us create the *actors*, ego vehicle and RSUs. We need the qualitative counterparts, the *actor categories*, and we will use the *actor categories* from the *scenario category*. Note that we can ignore the `uid` for now.

In [8]:
# Vehicles
EGO_VEHICLE = Actor(I2V_CATEGORY.actors[0],
                    name="ego vehicle",
                    initial_states=[State(StateVariable.LONGITUDINAL_ROAD_POSITION, [1, 0])],
                    desired_states=[State(StateVariable.LONGITUDINAL_ROAD_POSITION, [1, 400]),
                                    State(StateVariable.LATERAL_ROAD_POSITION, [-1, 0])],
                    properties=dict(color="blue"))

# Road side units
RSU_1 = Actor(I2V_CATEGORY.actors[1], name="RSU_1",
              properties=dict(version="2.0", securityStatus="disabled"))
RSU_2 = Actor(I2V_CATEGORY.actors[1], name="RSU_2",
              properties=dict(version="2.0", securityStatus="disabled"))

### 3.2 Create *activities* and *events*

As with the actors, we use the *activity categories* as defined in the *scenario category*. 

To instantiate an *activity*, the following arguments can be used:
- `category`: The corresponding *activity category* that we defined above.
- `parameters`: A dictionary with the parameters that are used to evaluate the model that has been defined in the corresponding *activity category*. To see which parameters are required, type `help(model)`, where `model` is to be replaced by the actual model. For example, `help(Sinusoidal)` will print the documentation of the `Sinusoidal` model.
- `start`: The starting event of the activity. Alternatively, instead of an `Event`, a single value can be passed. In that case, it is assumed that the activity starts when the time equals the provided value. In that case, an *event* is created at that particular time.
- `end`: The end event. Just as with `start`, a single value can be provided in which case an *event* is automatically instantiated.
- `name` (optional): You can give the activity a meaningful name.
- `uid` (optional): If not given, a (random) unique ID will be created.
- `tags` (optional): If not given, an empty list is assumed. Note that the `get_tags()` functionality works the same as for the *actors*, meaning that also the tags of the *activity category* are returned.

In [9]:
CRUISING_EGO1 = Activity(I2V_CATEGORY.activities[1], 
                         dict(xstart=22, xend=22), 
                         start=0, end=5, 
                         name="Initial cruising ego")
ACCELERATING_EGO = Activity(I2V_CATEGORY.activities[1], 
                            dict(xstart=22, xend=28),
                            start=CRUISING_EGO1.end, end=10,
                            name="Accelerating ego",
                            tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Accelerating])
CRUISING_EGO2 = Activity(I2V_CATEGORY.activities[1], dict(xstart=28, xend=28),
                         start=ACCELERATING_EGO.end, end=15,
                         name="Final cruising ego")
STRAIGHT_EGO = Activity(I2V_CATEGORY.activities[0], dict(xstart=[-1, 0]),
                        start=CRUISING_EGO1.start, end=CRUISING_EGO2.end,
                        name="Following lane ego")

# Define message parameters according to OpenSCENARIO elements and attributes
PARAMETERS = RSU_MESSAGES.model.get_pars(message_name="I2V_SpeedLimit", message="80 kph", 
                                         network=RSU_1,
                                         network_quality=dict(Speed=dict(connectionSpeed=5), 
                                                              Latency=dict(averageDelay=50), 
                                                              Reliability=dict(factor=0.9)))
SPEED_MESSAGE_RSU_1 = Activity(I2V_CATEGORY.activities[2],
                               PARAMETERS, start=CRUISING_EGO1.start, end=4,
                               name="Initial speed limit message",
                               tags=[Tag.Communication_I2V])

# Define message parameters according to OpenSCENARIO elements and attributes
PARAMETERS = RSU_MESSAGES.model.get_pars(message_name="I2V_SpeedLimit", message="80 kph", 
                                         network=RSU_1,
                                         network_quality=dict(Speed=dict(connectionSpeed=1), 
                                                              Latency=dict(averageDelay=100), 
                                                              Reliability=dict(factor=0.5)))
SPEED_MESSAGE_RSU_1_LOW_QUALITY = Activity(I2V_CATEGORY.activities[2],
                                           PARAMETERS, start=SPEED_MESSAGE_RSU_1.end, end=5,
                                           name="Low quality initial speed limit message",
                                           tags=[Tag.Communication_I2V])

# Define message parameters according to OpenSCENARIO elements and attributes
PARAMETERS = RSU_MESSAGES.model.get_pars(message_name="I2V_SpeedLimit", message="100 kph", 
                                         network=RSU_2,
                                         network_quality=dict(Speed=dict(connectionSpeed=8), 
                                                              Latency=dict(averageDelay=50), 
                                                              Reliability=dict(factor=0.8)))
SPEED_MESSAGE_RSU_2 = Activity(I2V_CATEGORY.activities[2], PARAMETERS,
                               start=SPEED_MESSAGE_RSU_1_LOW_QUALITY.end, end=STRAIGHT_EGO.end,
                               name="Final speed limit message",
                               tags=[Tag.Communication_I2V])

In [10]:
# Show parameters of the activity SPEED_MESSAGE_1_LOW_QUALITY:
SPEED_MESSAGE_RSU_1_LOW_QUALITY.parameters

{'message_name': 'I2V_SpeedLimit',
 'message': '80 kph',
 'frequency': 20,
 'receive': True,
 'network': 248376368610223481335686342718805772216,
 'network_quality': {'Speed': {'connectionSpeed': 1},
  'Latency': {'averageDelay': 100},
  'Reliability': {'factor': 0.5}}}

### 3.3 Create *physical things*

Just as with the *scenario category*, we only consider one *physical thing* that describes the road layout. For simplicitly, we assume a straight motorway road with 2 lanes of 3 m. The road is 400 m long.

In [11]:
MOTORWAY = PhysicalElement(
    I2V_CATEGORY.physical_elements[0], name="Motorway",
    properties=dict(Country="The Netherlands",
                    Roads=[dict(ID=1, Length=400, Type="motorway",
                                Geometry=[dict(s=0, xstart=0, ystart=0, heading=0, length=400,
                                               shape="line", parameters={})],
                                Lanes=[dict(ID=0, Width=0, Type='border', Marker='solid'),
                                       dict(ID=-1, Width=3, Type='driving', Marker='broken'),
                                       dict(ID=-2, Width=3, Type='driving', Marker='solid')],
                                Objects=dict(), Signals=dict())]))

## 4 Instantiate a *scenario*

Instantiating a *scenario* is very similar to instantiating a *scenario category*. One of the difference is that we need to provide the events that mark the start and the end of the scenario. After creating the *scenario*, we can define the physical things, actors, activities, and acts using the functions `set_physical_things`, `set_actors`, `set_activities`, and `set_acts`, such as we did for the *scenario category* using the respective qualitative descriptions.

In [12]:
I2V = Scenario(start=CRUISING_EGO1.start, end=CRUISING_EGO2.end, name="I2V scenario")
I2V.set_physical_elements([MOTORWAY])
I2V.set_actors([EGO_VEHICLE, RSU_1, RSU_2])
I2V.set_activities([CRUISING_EGO1, ACCELERATING_EGO, CRUISING_EGO2, STRAIGHT_EGO,
                    SPEED_MESSAGE_RSU_1, SPEED_MESSAGE_RSU_1_LOW_QUALITY, SPEED_MESSAGE_RSU_2])
I2V.set_acts([(EGO_VEHICLE, CRUISING_EGO1),
              (EGO_VEHICLE, ACCELERATING_EGO),
              (EGO_VEHICLE, CRUISING_EGO2),
              (EGO_VEHICLE, STRAIGHT_EGO),
              (EGO_VEHICLE, SPEED_MESSAGE_RSU_1),
              (EGO_VEHICLE, SPEED_MESSAGE_RSU_1_LOW_QUALITY),
              (EGO_VEHICLE, SPEED_MESSAGE_RSU_2)])

We can now see if the scenario is comprised by the scenario category.

In [13]:
I2V_CATEGORY.comprises(I2V)

True

## 5 Store *scenario*

Before the end of this tutorial, let's save the scenario.

In [14]:
# Let's save the scenario as json format.
FILENAME_I2V = os.path.join("examples", "i2v_quantitative.json")
with open(FILENAME_I2V, "w") as FILE:
    json.dump(I2V.to_json_full(), FILE, indent=4)

You reached the end of the fifth and final tutorial. We hope it has been useful! If you have any questions/requests, please contact us.